In [66]:
import pandas as pd
import os
import sys
import subprocess

### Paths

In [67]:
varicarta_path = "/private10/Projects/Nave_Oded_Project/resources/DBs/hg38/variants/Varicarta/variants_table.tsv"
variants_table_path = "/private10/Projects/Nave_Oded_Project/extended_variants_table/2nd_run.csv"

In [68]:
varicarta_df = pd.read_csv(varicarta_path, sep="\t", low_memory=False)
print(f"Varicarta table shape: {varicarta_df.shape}")


Varicarta table shape: (525687, 54)


In [69]:
# Varicarta/funcs.py
def deduplicate(df: pd.DataFrame) -> pd.DataFrame:
    df = df.copy()
    df['pos'] = df['pos'].astype(str)
    # Strip leading "I:" from alt (same as the shell script)
    df['alt'] = df['alt'].astype(str).str.replace(r'^I:', '', regex=True)

    # Keep only key columns before liftOver
    df = df[['chr', 'pos', 'ref', 'alt']].copy()

    # Match VEP awk guards: numeric pos, non-empty ref/alt
    df = df[df['pos'].str.match(r'^[0-9]+$')]
    df = df[df['ref'].notna() & df['alt'].notna() & (df['ref'] != "") & (df['alt'] != "")]

    # **Exact de-dup on the 4-key tuple** (what your VEP pipeline does)
    df = df.drop_duplicates(subset=['chr', 'pos', 'ref', 'alt'], keep='first')

    return df


In [70]:
    # Build keys from raw columns (as your current code does)
varicarta_df['chr'] = varicarta_df['chromosome'].astype(str)
varicarta_df['chr'] = 'chr' + varicarta_df['chr']

varicarta_df['pos'] = varicarta_df['start_hg19'].astype(str)

In [71]:
dedup_varicarta_df = deduplicate(varicarta_df)

In [72]:
print(f"Deduplicated Varicarta shape: {dedup_varicarta_df.shape}")
print(f"Number of duplicates removed: {varicarta_df.shape[0] - dedup_varicarta_df.shape[0]}")


Deduplicated Varicarta shape: (329439, 4)
Number of duplicates removed: 196248


In [73]:
variants_df = pd.read_csv(variants_table_path, low_memory=False)

In [74]:
variants_df = variants_df[variants_df["Varicarta"] == 1]

In [75]:
variants_df.shape

(365351, 119)

In [76]:
dedup_variants_df = deduplicate(variants_df)
print(f"Deduplicated variants_df shape: {dedup_variants_df.shape}")


Deduplicated variants_df shape: (329363, 4)


In [77]:
alternate_chr = dedup_variants_df[~dedup_variants_df['chr'].str.match(r'^(chr)?([1-9]|1[0-9]|2[0-2]|X|Y|MT)$')]
print(f"alternate_chr shape: {alternate_chr.shape}")

# remove alternate_chr from dedup_variants_df
dedup_variants_df = dedup_variants_df[~dedup_variants_df['chr'].isin(alternate_chr['chr'])]
print(f"dedup_variants_df shape after removing alternate_chr: {dedup_variants_df.shape}")

alternate_chr shape: (267, 4)
dedup_variants_df shape after removing alternate_chr: (329096, 4)


The minor difference between the two datasets is likely to unmapped variants in the liftover process. The Varicarta dataset was 

### Count unmapped variants

In [78]:
liftover_script = "/private10/Projects/Nave_Oded_Project/resources/DBs/hg38/variants/liftOverToHg38.sh"
liftover_chain = "/private10/Projects/Nave_Oded_Project/resources/DBs/hg38/.liftOver/hg19ToHg38.over.chain.gz"

In [79]:
def lift_over(df: pd.DataFrame) -> pd.DataFrame:
    """
    Function to perform liftOver from hg19 to hg38.
    This is a placeholder function; actual implementation would depend on the liftOver tool.
    """
    df['start'] = df['pos'].astype(int) - 1  # Convert pos to int and create start column
    df['end'] = df['start'] + df['ref'].str.len()  # Calculate end position based on ref length
    df = df[['chr', 'start', 'end', 'ref', 'alt']].copy()  # Reorder columns to match expected format
    # 5) Convert 'chr' to string type
    df['chr'] = df['chr'].astype(str)
    # 6) Convert 'start' and 'end' to int type
    df['start'] = df['start'].astype(int)
    df['end'] = df['end'].astype(int)

    # Ensure the DataFrame has the expected columns
    expected_columns = ['chr', 'start', 'end', 'ref', 'alt']
    for col in expected_columns:
        if col not in df.columns:
            raise ValueError(f"Missing expected column: {col}")
    
    tmp_file_path = "tmp_lifted_over_variants.tsv"
    df.to_csv(tmp_file_path, sep='\t', index=False, header=False)  # Save to a temporary file for liftOver

    if os.path.exists(liftover_script):
        if os.path.exists(liftover_chain):
            try:
                result_path = subprocess.run(['bash', liftover_script, tmp_file_path, liftover_chain], check=True, capture_output=True, text=True).stdout.strip()
                if not result_path: 
                    raise ValueError("LiftOver script did not return a valid file path.")
                # run the liftOver script with the temporary file and chain file and save the returned file path

                print("LiftOver completed successfully.")
            except subprocess.CalledProcessError as e:
                print(f"Error during LiftOver: {e}")
        else:
            print(f"Chain file not found at {liftover_chain}")
    else:
        print(f"LiftOver script not found at {liftover_script}")

    # Load the lifted over variants back into a DataFrame
    lifted_df = pd.read_csv(result_path, sep='\t', header=None, names=expected_columns)
    os.remove(tmp_file_path)  # Clean up the temporary file
    os.remove(result_path)  # Clean up the result file
    # Ensure the lifted DataFrame has the expected columns
    for col in expected_columns:
        if col not in lifted_df.columns:
            raise ValueError(f"Lifted DataFrame is missing expected column: {col}")
        
    print(" DataFrame head:")
    print(df[['chr', 'start', 'end', 'ref', 'alt']].head())  # Debugging output
    print(" Lifted DataFrame head:")
    print(lifted_df[['chr', 'start', 'end', 'ref', 'alt']].head())  # Debugging output
    # Return the lifted DataFrame to the original format
    lifted_df['chr'] = lifted_df['chr'].str.replace('chr', '', regex=False)
    lifted_df['start'] = (lifted_df['start'] + 1).astype(str)  # Convert start back to 1-based index
    # rename 'start' to 'pos'
    lifted_df.rename(columns={'start': 'pos'}, inplace=True)

    return lifted_df[['chr', 'pos', 'ref', 'alt']].reset_index(drop=True)

In [80]:
dedup_varicarta_lifted = lift_over(dedup_varicarta_df)

LiftOver completed successfully.
 DataFrame head:
     chr      start        end ref alt
0   chr2   38975220   38975221   C  CC
1   chr1  169824926  169824927   A   G
2   chr5  133635406  133635407   T   C
3  chr10   55583160   55583161   T   G
4  chr15   28953438   28953439   G   A
 Lifted DataFrame head:
     chr      start        end ref alt
0   chr2   38748078   38748079   C  CC
1   chr1  169855785  169855786   A   G
2   chr5  134299715  134299716   T   C
3  chr10   53823400   53823401   T   G
4  chr15   28708292   28708293   G   A


In [81]:
print(f"Number of unmapped coordinates: {dedup_varicarta_df.shape[0] - dedup_varicarta_lifted.shape[0]}")

Number of unmapped coordinates: 71


In [82]:
dedup_varicarta_lifted.shape

(329368, 4)

In [83]:
# print rows where the chr col is not number between 1-22 or X or Y or MT
alternate_chr = dedup_varicarta_lifted[~dedup_varicarta_lifted['chr'].str.match(r'^(chr)?([1-9]|1[0-9]|2[0-2]|X|Y|MT)$')]
print(f"alternate_chr shape: {alternate_chr.shape}")

# remove alternate_chr from dedup_varicarta_lifted
dedup_varicarta_lifted = dedup_varicarta_lifted[~dedup_varicarta_lifted['chr'].isin(alternate_chr['chr'])]
print(f"dedup_varicarta_lifted shape after removing alternate_chr: {dedup_varicarta_lifted.shape}")
print(f"Number of alternate chr removed: {alternate_chr.shape[0]}")

alternate_chr shape: (267, 4)
dedup_varicarta_lifted shape after removing alternate_chr: (329101, 4)
Number of alternate chr removed: 267


In [84]:
print(f"Difference between varicarta and variants table after liftOver: {dedup_varicarta_lifted.shape[0] - dedup_variants_df.shape[0]}")

Difference between varicarta and variants table after liftOver: 5


In [85]:
# save the differences in a new df
differences_df = pd.merge(dedup_varicarta_lifted, dedup_variants_df, on=['chr', 'pos', 'ref', 'alt'], how='outer', indicator=True)
differences_df = differences_df[differences_df['_merge'] != 'both']
differences_df = differences_df.drop(columns=['_merge'])
differences_df

,chr,pos,ref,alt
13148,1,215758592,T,T
169856,2,38748079,C,CC
281578,7,152230325,T,T
325288,X,141907819,TGCC,TGCC


In [117]:
varicarta_df[(varicarta_df['chr'] == 'chr1') & (varicarta_df['pos'] == "215931934") | (varicarta_df['chr'] == 'chr2') & (varicarta_df['pos'] == "38975221") | (varicarta_df['chr'] == 'chr7') & (varicarta_df['pos'] == "151927410") | (varicarta_df['chr'] == 'chrX') & (varicarta_df['pos'] == "140995605")]


,pid,paper_key,id,paper_id,sequencing_study_type,event_id,subject_id,sample_id,chromosome,start_hg19,...,LR_pred,VEST3_score,CADD_raw,CADD_phred,GERP_RS,phyloP46way_placental,phyloP100way_vertebrate,SiPhy_29way_logOdds,chr,pos
0,0,An2014,987929,0,exome,0,0,31.s1,2,38975221,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,chr2,38975221
252815,170,Hu2023,1616470,170,targeted,378264,54218,Hu2023:18,7,151927410,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,chr7,151927410
469104,190,Ben-Mahmoud2024,2019689,190,Genome Sequencing Identifies 13 Novel Candidat...,480361,62187,Ben-Mahmoud2024:12,1,215931934,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,chr1,215931934
469156,190,Ben-Mahmoud2024,2019741,190,Genome Sequencing Identifies 13 Novel Candidat...,480413,62225,Ben-Mahmoud2024:6,X,140995605,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,chrX,140995605


In [120]:
variants_df[(variants_df['chr'] == 'chr1') & (variants_df['pos'] == "215758592") | (variants_df['chr'] == 'chr2') & (variants_df['pos'] == "38748079") | (variants_df['chr'] == 'chr7') & (variants_df['pos'] == "152230325") | (variants_df['chr'] == 'chrX') & (variants_df['pos'] == "141907819")]

,chr,pos,ref,alt,AF,AFR_AF,AMR_AF,APPRIS,Allele,Amino_acids,...,gnomADg_FIN_AF,gnomADg_MID_AF,gnomADg_NFE_AF,gnomADg_REMAINING_AF,gnomADg_SAS_AF,miRNA,pLI_gene_value,is_ADAR_fixable,is_APOBEC_fixable,hg38
